# List of Exposures  at USDF in OGA


- work with Weakly_2024_17
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/09/13
- last update : 2024/05/02



In [1]:
import sys
print(sys.executable)

/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/bin/python3


In [2]:
print(sys.path)

['/sdf/data/rubin/user/dagoret/AuxTelComm/notebooks_usdf/butlertools', '/home/d/dagoret/repos/repos_w_2024_16/Spectractor/lib/python', '/home/d/dagoret/repos/repos_w_2024_16/atmospec/python', '/home/d/dagoret/repos/repos_w_2024_16/analysis_tools/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/summit_extras/g83f1fb8f92+e9fa2e932c/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/summit_utils/gbd71fbde96+d024441011/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/display_astrowidgets/g2bbee38e9b+d6b5cd48b5/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/cell_coadds/g4da81e4fa4+4c30af3487/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/gbdes/g73f919205b+ca4f911437/build/pydir', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linux64/drp_tasks/g3b9ebb4159+e4fcc55755/python', '/opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-8.0.0/Linu

In [3]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_16/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_16/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_16/Spectractor 	setup


In [4]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

Weekly 2024_16
   g4213664e8e+c2911b1728 	current w_2024_16 setup


In [5]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.io import fits
from astropy.time import Time

In [6]:
from IPython.display import display, HTML

In [7]:
#import lsst.afw.display as afwDisplay
#afwDisplay.setDefaultBackend('matplotlib')

In [8]:
import lsst.daf.butler as dafButler

In [9]:
#repo = "/sdf/group/rubin/repo/main"
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [10]:
collection='LATISS/raw/all'

## List of Exposures

In [11]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [12]:
print(butler.registry.dimensions["exposure"].RecordClass.fields)

exposure: 
  instrument: str
  id: int
  day_obs: int
  group: str
  physical_filter: str
  obs_id: str
  exposure_time: float
  dark_time: float
  observation_type: str
  observation_reason: str
  seq_num: int
  seq_start: int
  seq_end: int
  target_name: str
  science_program: str
  tracking_ra: float
  tracking_dec: float
  sky_angle: float
  azimuth: float
  zenith_angle: float
  has_simulated: bool
  can_see_sky: bool
  timespan: lsst.daf.butler.Timespan


In [13]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where= "instrument='LATISS'")):
    
    
    try:
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,pd.to_datetime(info.timespan.begin.to_string()),pd.to_datetime(info.timespan.end.to_string()) ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info.timespan.begin.jd,info.timespan.begin.mjd ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd, info_timespan_begin_mjd  ]
 
    
    if count < 5:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)
        
        mjd = Time(info.timespan.begin.to_string()).mjd
        jd = Time(info.timespan.begin.to_string()).jd
        print(mjd,jd)
    

----------------------------------------------------- 0 ---------------------------------------------------------
exposure:
  instrument: 'LATISS'
  id: 2023011100002
  day_obs: 20230111
  group: '2023-01-12T04:10:45.662'
  physical_filter: 'BG40_65mm_1~empty'
  obs_id: 'AT_O_20230111_000002'
  exposure_time: 5.0
  dark_time: 5.23673
  observation_type: 'engtest'
  observation_reason: 'engtest'
  seq_num: 2
  seq_start: 2
  seq_end: 2
  target_name: 'azel_target'
  science_program: 'unknown'
  tracking_ra: 307.79861942556755
  tracking_dec: -20.253626608705332
  sky_angle: 100.23298943653465
  azimuth: 0.0013227050221757963
  zenith_angle: 9.999793858936755
  has_simulated: False
  can_see_sky: None
  timespan: Timespan(begin=astropy.time.Time('2023-01-12 04:10:47.090019', scale='tai', format='iso'), end=astropy.time.Time('2023-01-12 04:10:52.327000', scale='tai', format='iso'))
	 id:                   2023011100002
	 day_obs:              20230111
	 seq_num:              2
	 type-of-o

In [14]:
info

exposure.RecordClass(instrument='LATISS', id=2024031300005, day_obs=20240313, group='2024031300005', physical_filter='unknown~unknown', obs_id='AT_O_20240313_000005', exposure_time=0.0, dark_time=0.00866342, observation_type='bias', observation_reason='bias', seq_num=5, seq_start=5, seq_end=5, target_name='UNKNOWN', science_program='unknown', tracking_ra=None, tracking_dec=None, sky_angle=None, azimuth=None, zenith_angle=None, has_simulated=False, can_see_sky=False, timespan=Timespan(begin=astropy.time.Time('2024-03-13 12:13:42.007008', scale='tai', format='iso'), end=astropy.time.Time('2024-03-13 12:13:42.016000', scale='tai', format='iso')))

In [15]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2023011100002,AT_O_20230111_000002,20230111,2,2023-01-12 04:10:47.090019,2023-01-12 04:10:52.327,engtest,azel_target,BG40_65mm_1~empty,9.999794,5.0,307.798619,-20.253627,100.232989,0.001323,9.999794,unknown,2.459957e+06,59956.174156
1,2023011200064,AT_O_20230112_000064,20230112,64,2023-01-13 04:46:30.894002,2023-01-13 04:46:31.920,engtest,FlatField position,BG40_65mm_1~empty,51.125506,0.8,51.903967,-67.247896,286.766848,205.698886,51.125506,DayTime_Checkout,2.459958e+06,59957.198969
2,2023011200063,AT_O_20230112_000063,20230112,63,2023-01-13 04:45:59.695981,2023-01-13 04:46:00.927,engtest,FlatField position,BG40_65mm_1~empty,51.125378,1.0,51.774217,-67.247837,286.766983,205.698885,51.125378,DayTime_Checkout,2.459958e+06,59957.198608
3,2023011200108,AT_O_20230112_000108,20230112,108,2023-01-13 05:04:31.147004,2023-01-13 05:04:32.180,flat,FlatField position,BG40_65mm_1~empty,51.126408,0.8,None,None,None,205.698861,51.126408,unknown,2.459958e+06,59957.211472
4,2023011200110,AT_O_20230112_000110,20230112,110,2023-01-13 05:04:38.360000,2023-01-13 05:04:39.397,flat,FlatField position,BG40_65mm_1~empty,51.126411,0.8,None,None,None,205.698861,51.126411,unknown,2.459958e+06,59957.211555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125509,2024031200457,AT_O_20240312_000457,20240312,457,2024-03-13 09:34:06.072990,2024-03-13 09:34:06.079,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398681
125510,2024031200459,AT_O_20240312_000459,20240312,459,2024-03-13 09:34:11.075980,2024-03-13 09:34:11.082,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398739
125511,2024031200461,AT_O_20240312_000461,20240312,461,2024-03-13 09:34:16.075993,2024-03-13 09:34:16.082,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398797
125512,2024031300003,AT_O_20240313_000003,20240313,3,2024-03-13 12:13:36.983016,2024-03-13 12:13:36.992,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.509456


In [16]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

In [17]:
df_exposure

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
0,2023011100002,AT_O_20230111_000002,20230111,2,2023-01-12 04:10:47.090019,2023-01-12 04:10:52.327,engtest,azel_target,BG40_65mm_1~empty,9.999794,5.0,307.798619,-20.253627,100.232989,0.001323,9.999794,unknown,2.459957e+06,59956.174156
1,2023011200064,AT_O_20230112_000064,20230112,64,2023-01-13 04:46:30.894002,2023-01-13 04:46:31.920,engtest,FlatField position,BG40_65mm_1~empty,51.125506,0.8,51.903967,-67.247896,286.766848,205.698886,51.125506,DayTime_Checkout,2.459958e+06,59957.198969
2,2023011200063,AT_O_20230112_000063,20230112,63,2023-01-13 04:45:59.695981,2023-01-13 04:46:00.927,engtest,FlatField position,BG40_65mm_1~empty,51.125378,1.0,51.774217,-67.247837,286.766983,205.698885,51.125378,DayTime_Checkout,2.459958e+06,59957.198608
3,2023011200108,AT_O_20230112_000108,20230112,108,2023-01-13 05:04:31.147004,2023-01-13 05:04:32.180,flat,FlatField position,BG40_65mm_1~empty,51.126408,0.8,None,None,None,205.698861,51.126408,unknown,2.459958e+06,59957.211472
4,2023011200110,AT_O_20230112_000110,20230112,110,2023-01-13 05:04:38.360000,2023-01-13 05:04:39.397,flat,FlatField position,BG40_65mm_1~empty,51.126411,0.8,None,None,None,205.698861,51.126411,unknown,2.459958e+06,59957.211555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125509,2024031200457,AT_O_20240312_000457,20240312,457,2024-03-13 09:34:06.072990,2024-03-13 09:34:06.079,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398681
125510,2024031200459,AT_O_20240312_000459,20240312,459,2024-03-13 09:34:11.075980,2024-03-13 09:34:11.082,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398739
125511,2024031200461,AT_O_20240312_000461,20240312,461,2024-03-13 09:34:16.075993,2024-03-13 09:34:16.082,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.398797
125512,2024031300003,AT_O_20240313_000003,20240313,3,2024-03-13 12:13:36.983016,2024-03-13 12:13:36.992,bias,UNKNOWN,unknown~unknown,None,0.0,None,None,None,None,None,unknown,2.460383e+06,60382.509456


## Selection of science

In [18]:
df_science = df_exposure[df_exposure.type == 'science']

In [19]:
df_science.sort_values(by="id",ascending=True,inplace=True)

/tmp/ipykernel_5825/3986782663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_science.sort_values(by="id",ascending=True,inplace=True)


In [20]:
df_science.head(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
124235,2022091200104,AT_O_20220912_000104,20220912,104,2022-09-13 01:20:16.390009,2022-09-13 01:20:46.618,science,HD167060,empty~ronchi170lpmm,35.761102,30.0,274.485264,-61.742148,130.613923,200.833163,35.761102,unknown,2.459836e+06,59835.055745
124234,2022091200105,AT_O_20220912_000105,20220912,105,2022-09-13 01:20:59.277003,2022-09-13 01:21:29.513,science,HD167060,empty~ronchi170lpmm,35.816568,30.0,274.485275,-61.742168,130.613889,200.943213,35.816568,unknown,2.459836e+06,59835.056242
115615,2022091200106,AT_O_20220912_000106,20220912,106,2022-09-13 01:21:48.291985,2022-09-13 01:22:18.520,science,HD167060,empty~holo4_003,35.884569,30.0,274.485183,-61.74221,130.613866,201.067976,35.884569,unknown,2.459836e+06,59835.056809
115616,2022091200107,AT_O_20220912_000107,20220912,107,2022-09-13 01:22:31.094006,2022-09-13 01:23:01.329,science,HD167060,empty~holo4_003,35.940561,30.0,274.485217,-61.74215,130.613861,201.176876,35.940561,unknown,2.459836e+06,59835.057304
124237,2022091200111,AT_O_20220912_000111,20220912,111,2022-09-13 01:24:55.866989,2022-09-13 01:25:26.113,science,HD2811,empty~ronchi170lpmm,52.803456,30.0,7.828469,-43.566523,351.230009,123.786942,52.803456,unknown,2.459836e+06,59835.058980
124236,2022091200112,AT_O_20220912_000112,20220912,112,2022-09-13 01:25:38.685988,2022-09-13 01:26:08.920,science,HD2811,empty~ronchi170lpmm,52.677652,30.0,7.828567,-43.566534,351.229973,123.762716,52.677652,unknown,2.459836e+06,59835.059476
115618,2022091200113,AT_O_20220912_000113,20220912,113,2022-09-13 01:26:27.713000,2022-09-13 01:26:57.947,science,HD2811,empty~holo4_003,52.535234,30.0,7.828656,-43.566495,351.229866,123.734754,52.535234,unknown,2.459836e+06,59835.060043
115617,2022091200114,AT_O_20220912_000114,20220912,114,2022-09-13 01:27:10.524999,2022-09-13 01:27:40.765,science,HD2811,empty~holo4_003,52.40619,30.0,7.828528,-43.566516,351.229887,123.710487,52.40619,unknown,2.459836e+06,59835.060538
124238,2022091200119,AT_O_20220912_000119,20220912,119,2022-09-13 01:29:43.197999,2022-09-13 01:30:13.443,science,HD185975,empty~ronchi170lpmm,57.111008,30.0,308.018105,-87.47647,85.868273,179.55936,57.111008,unknown,2.459836e+06,59835.062306
124239,2022091200120,AT_O_20220912_000120,20220912,120,2022-09-13 01:30:26.020016,2022-09-13 01:30:56.245,science,HD185975,empty~ronchi170lpmm,57.109986,30.0,308.018207,-87.47646,85.868022,179.568978,57.109986,unknown,2.459836e+06,59835.062801


In [21]:
df_science.tail(60)

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
40661,2024072900479,AT_O_20240729_000479,20240729,479,2024-07-30 03:15:28.115988,2024-07-30 03:15:58.343,science,Photo1800-1_008,SDSSr_65mm~empty,14.717857,30.0,270.262569,-25.041267,359.999843,286.91348,14.717857,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.135742
72274,2024072900480,AT_O_20240729_000480,20240729,480,2024-07-30 03:16:14.814990,2024-07-30 03:16:45.058,science,Photo1800-1_008,SDSSz_65mm~empty,14.880154,30.0,270.262672,-25.041284,359.999793,286.629412,14.880154,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.136283
76539,2024072900481,AT_O_20240729_000481,20240729,481,2024-07-30 03:17:02.924003,2024-07-30 03:17:33.157,science,Photo1800-1_008,empty~SDSSy_65mm,15.04623,30.0,270.262678,-25.041302,359.9997,286.344416,15.04623,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.136839
6382,2024072900482,AT_O_20240729_000482,20240729,482,2024-07-30 03:18:35.505999,2024-07-30 03:19:05.746,science,Photo1800-1_009,SDSSg_65mm~empty,15.475406,30.0,270.148414,-25.011733,359.999628,285.752917,15.475406,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.137911
40662,2024072900483,AT_O_20240729_000483,20240729,483,2024-07-30 03:19:22.330006,2024-07-30 03:19:52.559,science,Photo1800-1_009,SDSSr_65mm~empty,15.638795,30.0,270.148359,-25.011719,359.999677,285.489933,15.638795,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.138453
72275,2024072900484,AT_O_20240729_000484,20240729,484,2024-07-30 03:20:08.851983,2024-07-30 03:20:39.092,science,Photo1800-1_009,SDSSz_65mm~empty,15.798812,30.0,270.148419,-25.011707,359.999727,285.236983,15.798812,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.138991
76540,2024072900485,AT_O_20240729_000485,20240729,485,2024-07-30 03:20:57.212010,2024-07-30 03:21:27.452,science,Photo1800-1_009,empty~SDSSy_65mm,15.969502,30.0,270.148432,-25.01173,359.999662,284.971885,15.969502,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.139551
6383,2024072900502,AT_O_20240729_000502,20240729,502,2024-07-30 03:33:59.506017,2024-07-30 03:34:29.740,science,Photo1800-1_010,SDSSg_65mm~empty,18.704311,30.0,270.164698,-25.011722,359.999592,281.290365,18.704311,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.148605
40679,2024072900503,AT_O_20240729_000503,20240729,503,2024-07-30 03:34:46.182006,2024-07-30 03:35:16.421,science,Photo1800-1_010,SDSSr_65mm~empty,18.86715,30.0,270.164694,-25.011717,359.999595,281.098829,18.86715,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.149146
72276,2024072900504,AT_O_20240729_000504,20240729,504,2024-07-30 03:35:32.771011,2024-07-30 03:36:03.006,science,Photo1800-1_010,SDSSz_65mm~empty,19.033587,30.0,270.164736,-25.011749,359.999439,280.905601,19.033587,AUXTEL_PHOTO_IMAGING,2.460522e+06,60521.149685


In [22]:
df_science.day_obs.unique()

array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720, 20230801, 20230802, 20230803, 20230815, 20230816,
       20230817, 20230829, 20230912, 20230913, 20230914, 20230926,
       20230927, 20230928, 20231010, 20231011, 20231107, 20231108,
       20231113, 20231114, 20231115, 20231116, 20231120, 20231121,
       20231127, 20231128, 20231129, 20231130, 20231204, 20231205,
       20231206, 20231207, 20231211, 20231212, 20231214, 20231215,
       20231219, 20231220, 20231221, 20231222, 20240129, 20240

## Selection of date


  array([20220912, 20220913, 20220914, 20220927, 20220928, 20220929,
       20221011, 20221012, 20221013, 20221025, 20221026, 20221027,
       20221108, 20221109, 20221110, 20221122, 20221123, 20221124,
       20221207, 20221208, 20221209, 20221210, 20221212, 20230117,
       20230118, 20230119, 20230131, 20230201, 20230202, 20230214,
       20230215, 20230216, 20230228, 20230301, 20230302, 20230314,
       20230315, 20230316, 20230509, 20230510, 20230511, 20230523,
       20230524, 20230525, 20230704, 20230705, 20230706, 20230718,
       20230720, 20230801, 20230802, 20230803, 20230815, 20230816,
       20230817, 20230829, 20230912, 20230913, 20230914, 20230926,
       20230927, 20230928, 20231010, 20231011, 20231107, 20231108,
       20231113, 20231114, 20231115, 20231116, 20231120, 20231121,
       20231127, 20231128, 20231129, 20231130, 20231204, 20231205,
       20231206, 20231207, 20231211, 20231212, 20231214, 20231215,
       20231219, 20231220, 20231221, 20231222, 20240129, 20240130,
       20240131, 20240205, 20240206, 20240207, 20240208, 20240212,
       20240213, 20240214, 20240215, 20240219, 20240220, 20240222,
       20240223, 20240226, 20240227, 20240228, 20240229, 20240301,
       20240302, 20240303, 20240304, 20240305, 20240306, 20240307,
       20240308, 20240311, 20240312, 20240313, 20240314, 20240319,
       20240321, 20240325, 20240326, 20240327, 20240328, 20240329,
       20240401, 20240402, 20240403, 20240404, 20240405, 20240408,
       20240409, 20240410, 20240416, 20240417, 20240418, 20240419,
       20240422, 20240423, 20240424, 20240506, 20240527, 20240528,
       20240530, 20240624, 20240625, 20240626, 20240708, 20240709,
       20240710, 20240711, 20240715, 20240716, 20240717, 20240722,
       20240724, 20240729])

### Add those

20220504 - Not existing

In [162]:

# Januaryr 2024
#date_sel = 20240129
#date_sel = 20240130
#date_sel = 20240131
#date_sel = 20230928
date_sel = 20240717

## Query Dimension records on Exposures

In [163]:
where_clause = f"instrument=\'LATISS\' AND exposure.day_obs={date_sel} AND exposure.science_program=\'spec\'"

In [164]:
results_exposures = registry.queryDimensionRecords('exposure',datasets="raw",collections = 'LATISS/raw/all',where=where_clause)

## Select date

In [165]:
df_science_selected = df_science[df_science.day_obs == date_sel ]

In [166]:
len(df_science_selected)

230

# List of filters

In [167]:
list_of_filters = df_science_selected['filter'].unique()
list_of_filters

array(['empty~holo4_003', 'empty~blue300lpmm_qn1'], dtype=object)

In [168]:
selected_filters = []
for filt in list_of_filters:
    #flag_sel = (filt.find('holo4') != -1) or (filt.find('ronchi90lpmm') != -1) or (filt.find('ronchi170lpmm') != -1)
    flag_sel = (filt.find('holo4') != -1) 
    #flag_sel = (filt.find('ronchi170') != -1) 
    if flag_sel:
        selected_filters.append(filt) 
selected_filters = np.array(selected_filters)
selected_filters       

array(['empty~holo4_003'], dtype='<U15')

## Selection of filter

- But now we have a loop on filters

In [169]:
#idx_sel_filt = 0
#selected_filter = selected_filters[0]
#selected_filter

## Final selection of exposure

- loop on selected filters

In [170]:
pd.set_option('display.max_rows', None)

In [171]:
all_def_science_selected_final = []

for selected_filter in selected_filters:

    print(selected_filter)
    df_science_selected_final = df_science_selected[df_science_selected["filter"] ==  selected_filter]
    
    #print(df_science_selected_final)
    all_def_science_selected_final.append(df_science_selected_final.copy())
    
    df = df_science_selected_final
    df_sel = df[['day_obs', 'seq_num']]
    
    
    NN = len(df)
    
    if NN>0:
        # build the line
        
        df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)
    
    
         # prepare output
        filename_out = "visitdispersers_"+ str(date_sel) + "_filt_" + selected_filter.replace('~','-') + ".list"
        print(filename_out)
        
        df_out = df_sel[["line"]]
        df_out.to_csv(filename_out, index=False, header=None)

empty~holo4_003
visitdispersers_20240717_filt_empty-holo4_003.list


/tmp/ipykernel_5825/3368292521.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sel['line'] = df_sel.apply(lambda row: str(row['day_obs'])  + " " + str(row['seq_num']),axis = 1)


In [172]:
N = len(all_def_science_selected_final)

In [173]:
if N>=1:
    n0=len(all_def_science_selected_final[0])
    display(all_def_science_selected_final[0])

,id,obs_id,day_obs,seq_num,time_start,time_end,type,target,filter,zenith_angle,expos,ra,dec,skyangle,azimuth,zenith,science_program,jd,mjd
110787,2024071700271,AT_O_20240717_000271,20240717,271,2024-07-17 23:18:30.919999,2024-07-17 23:19:31.151,science,HD111980,empty~holo4_003,22.539033,60.0,193.327055,-18.540089,215.688928,295.966889,22.539033,spec-survey,2.460509e+06,60508.971191
110788,2024071700272,AT_O_20240717_000272,20240717,272,2024-07-17 23:19:43.828998,2024-07-17 23:20:44.055,science,HD111980,empty~holo4_003,22.77627,60.0,193.327083,-18.540087,215.688955,295.539122,22.77627,spec-survey,2.460509e+06,60508.972035
110791,2024071700278,AT_O_20240717_000278,20240717,278,2024-07-17 23:27:55.841014,2024-07-17 23:28:56.075,science,HD185975,empty~holo4_003,59.872103,60.0,306.74372,-87.466951,358.108483,176.97689,59.872103,spec-survey,2.460509e+06,60508.977730
110792,2024071700279,AT_O_20240717_000279,20240717,279,2024-07-17 23:29:08.852004,2024-07-17 23:30:09.084,science,HD185975,empty~holo4_003,59.858222,60.0,306.743783,-87.466945,358.108279,176.975868,59.858222,spec-survey,2.460509e+06,60508.978575
110793,2024071700285,AT_O_20240717_000285,20240717,285,2024-07-17 23:38:23.581001,2024-07-17 23:38:53.823,science,HD146233,empty~holo4_003,31.4575,30.0,243.895613,-8.377995,313.386523,51.14854,31.4575,spec-survey,2.460509e+06,60508.984995
110794,2024071700286,AT_O_20240717_000286,20240717,286,2024-07-17 23:39:06.511003,2024-07-17 23:39:36.743,science,HD146233,empty~holo4_003,31.336864,30.0,243.895648,-8.378046,313.386765,50.898258,31.336864,spec-survey,2.460509e+06,60508.985492
110795,2024071700292,AT_O_20240717_000292,20240717,292,2024-07-17 23:44:36.877990,2024-07-17 23:45:07.113,science,HD177517,empty~holo4_003,61.022173,30.0,286.408002,-15.65879,333.715896,91.927161,61.022173,spec-survey,2.460509e+06,60508.989316
110796,2024071700293,AT_O_20240717_000293,20240717,293,2024-07-17 23:45:19.894011,2024-07-17 23:45:50.133,science,HD177517,empty~holo4_003,60.867142,30.0,286.407942,-15.658779,333.715833,91.83946,60.867142,spec-survey,2.460509e+06,60508.989814
110797,2024071700299,AT_O_20240717_000299,20240717,299,2024-07-17 23:50:49.346982,2024-07-17 23:51:19.580,science,HD146233,empty~holo4_003,29.428814,30.0,243.895605,-8.378972,309.511377,46.565238,29.428814,spec-survey,2.460509e+06,60508.993627
110798,2024071700300,AT_O_20240717_000300,20240717,300,2024-07-17 23:51:32.411001,2024-07-17 23:52:02.638,science,HD146233,empty~holo4_003,29.316419,30.0,243.895595,-8.378944,309.511345,46.28467,29.316419,spec-survey,2.460509e+06,60508.994125


In [78]:
if N>=2:
    n1=len(all_def_science_selected_final[1])
    display(all_def_science_selected_final[1])

In [37]:
if N>=3:
    n2=len(all_def_science_selected_final[2])
    display(all_def_science_selected_final[2])

In [38]:
if N>=4:
    n3=len(all_def_science_selected_final[3])
    display(all_def_science_selected_final[3])

In [39]:
if N>=5:
    n4=len(all_def_science_selected_final[4])
    display(all_def_science_selected_final[4])

In [40]:
df_forsave = pd.concat(all_def_science_selected_final)

In [41]:
filename_out_csv = "holosummary_"+ str(date_sel) + ".csv"
filename_out_excel = "holosummary_"+ str(date_sel) + ".xlsx"

In [42]:
filename_out_csv

'holosummary_20240729.csv'

In [43]:
df_forsave.to_csv(filename_out_csv) 

In [ ]:
#! pip install openpyxl

In [44]:
df_forsave.to_excel(filename_out_excel) 